In [4]:
!pip install -U -q google-genai

In [26]:
import os
from credentials import get_credentials_gemini
from google import genai
from google.genai import types
from IPython.display import display, Markdown
import pandas as pd

In [19]:
API_KEY = get_credentials_gemini()

client = genai.Client(api_key=API_KEY)




In [32]:
def get_completion(prompt: str) -> str:
    '''
    This function generates a completion for a given prompt using the Gemini 1.5 pro model.
    Parameters:
    prompt: The prompt to generate a completion for.
    Returns:
    The completion generated by the model.
    '''
    completion = client.models.generate_content(
        model="gemini-1.5-pro",
        contents=prompt,
        config=types.GenerateContentConfig(
        temperature= 1.7,
    ),
    )
    return completion.text

In [21]:
#Creat a path to access the data
DATAPATH = os.path.join('..', 'data')

#Read the statute and case summary
statute = open(os.path.join(DATAPATH, '18USC924c1.txt'), 'r').read()
case_summary = open(os.path.join(DATAPATH, 'smith_case_summary.txt'), 'r').read()


In [22]:
prompt = f'''
You are asked to consider a legal scenario from the perspective of an ordinary person, not a legal expert. Your task is to read a statute and a case summary, then decide whether the actions described constitute 'use' of a gun as defined by the statute.

First, please read the following statute:

<statute>
{statute}
</statute>

Now, read the following case summary:

<case_summary>
{case_summary}
</case_summary>

Before providing your final answer, please analyze the situation in detail inside <legal_interpretation> tags. Consider the facts of the case and how they relate to the language of the statute. Remember, you're approaching this as an average person, not a legal professional.

In your analysis, include the following:
1. Quote the relevant parts of the statute that define 'use' of a gun.
2. List the key actions Mr. Smith took with the gun, according to the case summary. Number each action.
3. Explain how an ordinary person might interpret these actions in relation to the statute.
4. Identify any aspects of the case that might influence your interpretation.
5. Present arguments for and against considering Mr. Smith's actions as 'use' of the gun.
6. Form your opinion on whether a reasonable person would consider that Mr. Smith 'used' his gun in the sense that the language of the statute requires.

Based on your analysis, please provide your answer in the following format:

ANSWER: [Your answer (yes or no)]
REASONING: [Your reasoning, explained in simple terms that an average person would use]

Please provide your response now.
'''

In [33]:
response = get_completion(prompt)

In [34]:
display(Markdown(response))

<legal_interpretation>
1. The relevant part of the statute defines "use" implicitly by connecting it to crimes of violence or drug trafficking: "Whoever, during and in relation to any crime of violence or drug trafficking crime... uses or carries a firearm..."  The statute doesn't specifically define "use," but it implies a connection to the underlying crime.


2. Key actions Mr. Smith took with the gun:
    1. Possessed the MAC-10 (brought it from Tennessee to Florida).
    2. Showed the MAC-10 to the undercover officer.
    3. Offered the MAC-10 in trade for cocaine.
    4. Pulled the MAC-10 out of a bag to show the undercover officer.



3. An ordinary person might interpret these actions as follows:  Mr. Smith clearly had the gun and intended to use it to acquire more drugs to sell. He was trying to trade it directly for cocaine, demonstrating it was integral to his drug trafficking operation.  He wasn’t just *carrying* the gun, he actively involved it in a proposed drug transaction.  


4. Influencing aspects of the case:  The fact that Mr. Smith traveled across state lines suggests a more serious and planned criminal enterprise. His modification of the weapon to be fully automatic also suggests he intended it to be a dangerous tool. He actively sought a drug trade specifically *using* the gun as barter.  The summary explicitly says the MAC-10 was a favorite of criminals and could be “devastating.”


5. Arguments:
    For "use": Mr. Smith took the gun out and displayed it as the central part of a drug deal negotiation. He was essentially trying to purchase cocaine with the gun itself, not with money.  It wasn’t passively in his possession; it was actively being bartered. 

    Against "use":  He didn't fire the weapon, nor did he explicitly threaten anyone with it.  One could argue he was simply *carrying* it and happened to mention it during the drug discussion.



6. Opinion:  A reasonable person would likely conclude Mr. Smith "used" the gun within the meaning of the statute. While he didn't fire it, his presentation of the weapon, his proposed barter of it for cocaine, and the context of the overall drug trafficking operation strongly link the gun to the criminal activity.  The gun wasn’t incidental; it was a key component in his attempted drug deal.


</legal_interpretation>

ANSWER: yes
REASONING: Mr. Smith didn’t just have the gun with him; he took it out and offered it as direct payment for drugs.  He was using the gun as a tool to further his drug dealing.  This seems like "use" given he was actively trying to trade the gun itself for drugs as a key part of a drug trafficking crime.


In [12]:
#Create a prompt with only answer and reasoning
new_prompt = f'''
You are asked to consider a legal scenario from the perspective of an ordinary person, not a legal expert. Your task is to read a statute and a case summary, then decide whether the actions described constitute 'use' of a gun as defined by the statute.

First, please read the following statute:

<statute>
{statute}
</statute>

Now, read the following case summary:

<case_summary>
{case_summary}
</case_summary>

Before providing your final answer, please analyze the situation in detail. Consider the facts of the case and how they relate to the language of the statute. Remember, you're approaching this as an average person, not a legal professional.

Specifically, consider the following (but DO NOT include them in your response):
1. The relevant parts of the statute that define 'use' of a gun.
2. The key actions Mr. Smith took with the gun, according to the case summary. Number each action.
3. How an ordinary person might interpret these actions in relation to the statute.
4. Any aspects of the case that might influence your interpretation.
5. Arguments for and against considering Mr. Smith's actions as 'use' of the gun.
6. Your opinion on whether a reasonable person would consider that Mr. Smith 'used' his gun in the sense that the language of the statute requires.

Based on your analysis, please provide your answer in the following format:

ANSWER: [Your answer (yes or no)]
REASONING: [Your reasoning, explained in simple terms that an average person would use]

Please provide your response now.
'''


In [35]:
test_response = get_completion(new_prompt)
display(Markdown(test_response))

ANSWER: Yes

REASONING:  Smith didn't just have the gun with him; he took it out, showed it off, and offered to trade it for drugs.  He was actively trying to use the gun as a bargaining chip to get something he wanted – cocaine.  To me, that goes beyond simply carrying a firearm. He was using it as part of a drug deal.


In [13]:
test_response[test_response.find("ANSWER:") + 7:test_response.find("REASONING:")].strip(), test_response[test_response.find("REASONING:") + 10:].strip()

('Yes',
 'Okay, so the law says if you "use" a gun during a crime, you get extra prison time. In this case, Smith was trying to trade his automatic weapon for drugs. He pulled the gun out of a bag and showed it to a guy, specifically offering it as payment for cocaine. To me, that’s definitely *using* the gun. He wasn’t just holding it; he was trying to use it as a tool in his drug deal. It\'s like using a credit card to buy something, but instead of a card, it\'s a gun. He was using it to get what he wanted: cocaine. The fact that the deal didn\'t actually go through doesn\'t change the fact that he was trying to use the gun to get drugs. So, yeah, he used it.')

In [36]:
#Test
test_responses = []
test_answers = []
test_reasoning = []

for i in range(3):
    t_response = get_completion(new_prompt)
    test_answers.append(t_response[t_response.find("ANSWER:") + 7:t_response.find("REASONING:")].strip())
    test_reasoning.append(t_response[t_response.find("REASONING:") + 10:].strip())
    test_responses.append(t_response)


# Convert to DataFrame
test_df = pd.DataFrame({'response': test_responses, 'answer': test_answers, 'reasoning': test_reasoning})

# Display the results
print("DataFrame Structure:")
display(test_df)

print("\nDetailed View of Each Response:")
for i, row in test_df.iterrows():
    print(f"\nTest {i+1}:")
    print(f"Raw Response: {row['response']}")
    print(f"Extracted Answer: {row['answer']}")
    print(f"Extracted Reasoning: {row['reasoning']}")
    print("-" * 50)

DataFrame Structure:


,response,answer,reasoning
0,ANSWER: Yes\n\nREASONING: Mr. Smith took the ...,Yes,Mr. Smith took the gun out of his bag and show...
1,ANSWER: Yes\n\nREASONING: Smith didn't just h...,Yes,Smith didn't just have the gun with him. He to...
2,ANSWER: Yes\n\nREASONING: Mr. Smith didn't ju...,Yes,Mr. Smith didn't just have the gun with him; h...



Detailed View of Each Response:

Test 1:
Raw Response: ANSWER: Yes

REASONING:  Mr. Smith took the gun out of his bag and showed it to the undercover officer as part of a proposed drug deal.  He wasn't just carrying it; he was actively using it as a bargaining chip, trying to trade it for cocaine. To me, that goes beyond simply having it with him.  He was using it as a tool to commit a crime, a drug trafficking crime specifically.

Extracted Answer: Yes
Extracted Reasoning: Mr. Smith took the gun out of his bag and showed it to the undercover officer as part of a proposed drug deal.  He wasn't just carrying it; he was actively using it as a bargaining chip, trying to trade it for cocaine. To me, that goes beyond simply having it with him.  He was using it as a tool to commit a crime, a drug trafficking crime specifically.
--------------------------------------------------

Test 2:
Raw Response: ANSWER: Yes

REASONING:  Smith didn't just have the gun with him. He took it out, showed it

In [37]:
#Create a sample of 100 responses and put them in a Pandas DataFrame
# Create empty lists to store answers and reasoning
answers = []
reasoning = []

# Generate 100 responses
for i in range(100):
    response = get_completion(new_prompt)
    
    # Extract answer and reasoning using simple string parsing
    # Assuming format is consistent with "ANSWER:" and "REASONING:" labels
    answer_start = response.find("ANSWER:") + 7
    reasoning_start = response.find("REASONING:") + 10
    
    answer = response[answer_start:response.find("REASONING:")].strip()
    reason = response[reasoning_start:].strip()
    
    answers.append(answer)
    reasoning.append(reason)

# Create DataFrame
responses_df = pd.DataFrame({
    'answer': answers,
    'reasoning': reasoning
})

# Display first few rows
display(responses_df.head())


,answer,reasoning
0,Yes,Smith didn't just have the gun with him. He to...
1,Yes,Smith didn't just have the gun with him. He to...
2,Yes,Smith didn't just have the gun with him; he to...
3,Yes,Mr. Smith didn't just have the gun with him; h...
4,Yes,Smith didn't just have the gun with him. He to...


In [38]:
#Count how many "yes" and "no" answers there are
print(responses_df['answer'].value_counts())

answer
Yes    100
Name: count, dtype: int64


In [39]:
#save the Dataframe to a csv file in the data folder
responses_df.to_csv(os.path.join(DATAPATH, 'smith_responses_gemini_temp_1.7.csv'), index=False)